# 날짜(DATE) 

In [1]:
from __future__ import unicode_literals
import pickle
import datetime
import random

In [2]:
import konlpy
from konlpy.utils import pprint

In [3]:
import re

In [4]:
TIME = {}

시간은 지금 or 오늘을 기준 0으로 잡고 offset을 저장하는 방식으로

### 지금(now) 

In [5]:
if '지금' in TIME.keys():
    offset = TIME['지금']
     # 시차 교정
    now = datetime.datetime.now()+ datetime.timedelta(hours=9)
    if offset == 0:        
        print(now)
        print(type(now.strftime('%Y-%m-%d')))

In [6]:
def ent_date(w,t,numflag,entities,sentence):

    nday = re.compile('[가-힣\s\d]+일')
    nweek = re.compile('[가-힣\s\d]+주')
    nmonth = re.compile('[가-힣\s\d]+달')
    ndate = re.compile('[가-힣\s\d]+월 [가-힣\s\d]+일')
    TIME={'지금':0, '오늘':0, '어제':-1,'내일':1,'내일모레':2,'모레':2,'그저께':-2}
    
    if (t == 'Noun') & (w in TIME.keys()):
            entities['DATE'] = TIME[w]
        
    if (t == 'Number'):
        if(ndate.match(sentence)!= None):
            findMonth = re.compile('[\d]+월')
            findDay = re.compile('[\d]일')
            month = findMonth.findall(sentence)[0][:-1]
            day = findDay.findall(sentence)[0][:-1]
            timevalue = datetime.date.today()-datetime.date.today().replace(month=int(month),day=int(day))
            if datetime.date.today() > datetime.date.today().replace(month=int(month),day=int(day)):
                entities['DATE'] = timevalue.days*-1
            else: entities['DATE'] = timevalue.days
        elif(nday.match(sentence) != None): 
            numflag = 1
            findDay = re.compile('[\d]+일')
            day = findDay.findall(sentence)[0][:-1]
            entities['DATE'] = numflag*int(day)
        elif(nweek.match(sentence) != None): 
            numflag = 7
            findWeek = re.compile('[\d]+주')
            week = findWeek.findall(sentence)[0][:-1]
            entities['DATE'] = numflag*int(week)
        elif(nmonth.match(sentence) != None): 
            numflag = 30
            findMonth = re.compile('[\d]+달')
            month = findMonth.findall(sentence)[0][:-1]
            entities['DATE'] = numflag*int(month)
        else: entities['DATE'] = 0
    
    if w =='저번':
        if (re.compile('저번주').match(sentence) != None) | (re.compile('저번 주').match(sentence) != None): entities['DATE'] = -7
        elif (re.compile('저번달').match(sentence) != None) | (re.compile('저번 달').match(sentence) != None): entities['DATE'] = -30
    
    if (w=='다음주'): entities['DATE'] = 7
    
    if (w =='다음') | (w=='담다'):
        # special token 
        if  (re.compile('다음 주').match(sentence) != None): entities['DATE'] = 7
        elif (re.compile('담주').match(sentence) != None): entities['DATE'] = 7
        elif (re.compile('담달').match(sentence) != None): entities['DATE'] = 30
        elif (re.compile('다음달').match(sentence) != None) | (re.compile('다음 달').match(sentence) != None): entities['DATE'] = 30
    
    if w =='이번':
        # special token 
        if (re.compile('이번주').match(sentence) != None) | (re.compile('이번 주').match(sentence) != None): entities['DATE'] = 'WEEK'
        elif (re.compile('저번달').match(sentence) != None) | (re.compile('저번 달').match(sentence) != None): entities['DATE'] = 'MONTH'
    
    
    if (numflag>0) & (w == '전'): 
        entities['DATE'] = entities['DATE']*-1
        
    return numflag, entities

## 테스트 

In [11]:
def disintegrate(sentence, needThese): 
    """sentence는 input문장, needThese 그 Intent에서 필요하는 ENTITIES 리스트"""
    disintegrated_sentence = konlpy.tag.Twitter().pos(sentence, norm=True, stem=True)    
    result = []
    entities = {}
    numflag = 0 # 1 7 30
    # needThese에 명시된 엔터티 딕셔너리 불러오기
    # entities 검사기는 needThese에 따라 on/off 하듯이?
    for w, t in disintegrated_sentence: 
        numflag, entities = ent_date(w,t,numflag,entities,sentence)
    
        
        if t not in ['Eomi', 'Josa', 'Number', 'KoreanParticle', 'Punctuation']:            
            result.append(w) 
    
    return ' '.join(result), entities

In [12]:
sentence, ent = disintegrate('오후에 날씨 알려줘!',['DATE']) # DAY 단위로만,,,,

In [13]:
pprint(ent)

{}


In [14]:
def getWeather(entities,location,_date=datetime.datetime.now()+ datetime.timedelta(hours=9)):
    if 'DATE' in entities.keys():
        temp_date = entities['DATE']
            
        try: 
            int(temp_date)
            if temp_date > 0: _date+=datetime.timedelta(days=temp_date)
            else: _date-=datetime.timedelta(days=abs(temp_date))
        except:
            "아직 구현 안됌.. 특정 기간동안의 날씨"
        
    templates=["%s의 %s 날씨는 맑아요!", "%s의 %s 날씨는 비가 오네요.."] # 이것도 구현 필요(날씨 api)
        
    result = random.choice(templates)
        
    return result % (location, _date.strftime('%Y-%m-%d'))

In [15]:
pprint(getWeather(ent,'수지'))

수지의 2016-10-29 날씨는 비가 오네요..


## 최종 테스트

In [16]:
sentence, ent = disintegrate('내일 날씨 좀 알려줘',['DATE']) # DAY 단위로만,,,,
pprint(getWeather(ent,'수지'))

수지의 2016-10-30 날씨는 비가 오네요..
